In [2]:
from random import randint
from math import sqrt
import arcpy

def sq_dist(p,q):
    return pow(p[0] - q[0],2) + pow(p[1] - q[1], 2)

def linear_search(points,query):
    neighbor_sqd = float('inf')
    for point in points:
        d = sq_dist(point,query)
        if d < neighbor_sqd:
            neighbor = point
            neighbor_sqd = d
    return neighbor

def neighbor_index(points,query):
    return points.index(linear_search(points,query))

def standardize(data):
    mx = float(sum([d[0] for d in data])) / len(data)
    my = float(sum([d[1] for d in data])) / len(data)
    sdx = sqrt(sum([pow((d[0] - mx),2) for d in data]) / len(data))
    sdy = sqrt(sum([pow((d[1] - my),2) for d in data]) / len(data))
    return [((d[0] - mx) / sdx,(d[1] - my) / sdy) for d in data]

def clustering(data,num,classification):
    centroids = []
    for c in range(num):
        members = [i for i in range(len(data)) if classification[i] == c]
        if len(members) > 0:
            sumx = float(sum([data[i][0] for i in members]))
            sumy = float(sum([data[i][1] for i in members]))
            centroids.append((sumx / len(members), sumy / len(members)))
    return centroids

def k_means(data,num):
    temporary = [randint(0,num - 1) for i in range(len(data))]
    classification = []
    while classification != temporary:
        classification = temporary
        centroids = clustering(data,num,classification)
        temporary = [neighbor_index(centroids,d) for d in data]
    return classification

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

fig = plt.figure(figsize = matplotlib.figure.figaspect(1))
plt.clf()
ax = fig.add_subplot(1,1,1)
ax.scatter([d[0] for d in data], [d[1] for d in data], s = 60, c = assignment)
plt.show

In [5]:
layer = arcpy.GetParameterAsText(0)
field1 = arcpy.GetParameterAsText(1)
field2 = arcpy.GetParameterAsText(2)
number = arcpy.GetParameter(3)

cursor = arcpy.SearchCursor(layer)
data = [(row.getValue(field1),row.getValue(field2)) for row in cursor]

assignment = k_means(standardize(data),int(number))

arcpy.DeleteField_management(layer,'cluster')
arcpy.AddField_management(layer,'cluster','SHORT',6)

i = 0
cursor = arcpy.UpdataCursor(layer)
for row in cursor:
    row.cluster = assignment[i]
    row.updateRow(row)
    i += 1

OSError: "" が存在しません